# Flask Integration

This jupyter notebook showcase an example on how to integrate your 'Agent' with a flask server.

Let's build an echo `Agent` returning the last message received.

First of all we need to define a function taking as input an `ExecuteRequest` and returning an `ExecuteResponse`.
We can then retrieve all the information we need from the request argument to build our response.

In [ ]:
from flask import Flask

from theoriq.execute import ExecuteRequest, ExecuteResponse
from theoriq.schemas import DialogItem, DialogItemBlock
from datetime import datetime, timezone

def echo_agent(req: ExecuteRequest) -> ExecuteResponse:
    # Retrieve the last message of the dialog
    dialog_item = req.body.items[-1]
    data = dialog_item.items[0].data
    
    # Let's return this data as our response
    response_body = DialogItem(
        timestamp=datetime.now(timezone.utc).isoformat(),
        source="My Echo Agent",
        sourceType="Agent",
        items=[DialogItemBlock(type="text", data=data)],
    )
    return ExecuteResponse(response_body)

Now that we have our 'execute' function ready, we can integrate it with the `theoriq` flask helpers to start a `theoriq` compliant flask server.

In [ ]:
# First let's create an AgentConfig
from theoriq.agent import AgentConfig
from biscuit_auth import KeyPair 
from theoriq.extra.flask import theoriq_blueprint

agent_config = AgentConfig(
    theoriq_public_key=KeyPair().public_key,
    agent_kp=KeyPair()
)

# Now we can use the flask helpers to create a 'BluePrint' with all the routes needed for our agent.
theoriq_routes = theoriq_blueprint(agent_config, echo_agent)

The created blueprint will contain the following endpoints:
- `/theoriq/api/v1alpha1/system/challenge` (See the challenge documentation for more explanation).
- `/theoriq/api/v1alpha1/execute` (Wrapper around the given 'execute' function).
- `/api/v1alpha1/behaviors/chat-completions` (Same functionality as the `execute` endpoint`; used for backward compatibility)

You can register this blueprint inside your flask application, and you'll be ready to go with the `theoriq` protocol!

In [ ]:
app = Flask(__name__)
app.register_blueprint(theoriq_routes)
app.run("0.0.0.0", 5757)